In [2]:
import pandas as pd
import numpy as np
from random import randrange
from math import sqrt

np.random.seed(0)

In [3]:
def splitting_data(df,no_data_div):######################################## dddddddddddddddddd  ###########
    n = int(len(df)/no_data_div)
    splitted_data = []
    copy_data = list(df)
    for i in range(no_data_div):
        local_lst = []
        while len(local_lst) < n:
            idx = randrange(len(copy_data))
            s = copy_data.pop(idx)
            local_lst.append(s)
        splitted_data.append(local_lst)
    #print(splitted_data)    
    return splitted_data    
           

In [4]:
def subsample(train,ssize):
    sample = []
    n_sample = round(len(train) * ssize)
    while len(sample) < n_sample:
        index = randrange(len(train))
        sample.append(train[index])
    return sample

In [5]:
def gini_index(groups, class_values):
    gini = 0.0
    for class_value in class_values:
        
        for group in groups:
            size = len(group)
            if size == 0:
                continue
            proportion = [row[-1] for row in group].count(class_value) / float(size)
            gini += (proportion * (1.0 - proportion))
    return gini

In [6]:
# splitting the dataset
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

In [7]:
def get_split(dataset, n_features):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    features = list()
    while len(features) < n_features:
        index = randrange(len(dataset[0])-1)
        if index not in features:
            features.append(index)
    for index in features:
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups

    return {'index':b_index, 'value':b_value, 'groups':b_groups}

In [8]:
# generating last node
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

In [9]:
# recursively splitting the nodes 
def split(node, max_depth, min_size, n_features, depth):

    left, right = node['groups']
    del(node['groups'])

    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return

    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return

    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, n_features)
        split(node['left'], max_depth, min_size, n_features, depth+1)

    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth+1)

In [10]:
# building the tree
def build_tree(data,max_depth, min_size,no_features):
    root = get_split(data, no_features)
    split(root, max_depth, min_size, no_features, 1)
    return root
    

In [11]:
# prediction
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [12]:
def predict_bagg(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    return max(set(predictions), key=predictions.count)

In [13]:
def Random_Forest(trainingset, testingset, max_depth, min_size, sample_size, x, no_features):
    trees = []
    predictions = []
    for i in range(x):
        data_ = subsample(trainingset,sample_size)
        tree = build_tree(data_,max_depth, min_size,no_features)
        trees.append(tree)
        
    predictions = [predict_bagg(trees, row) for row in testingset]    
    return(predictions)    
    

In [14]:
# Calculate accuracy
def checking(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [15]:
def evaluation(df, no_data_div, max_depth, min_size, sample_size, x, no_features):
    sub_samples = splitting_data(df,no_data_div)
    scores = []
    for sub_sample in sub_samples:
        trainingset = list(sub_samples)
        trainingset.remove(sub_sample)
        trainingset = sum(trainingset,[])
        
        testingset = []
        actual_label = []
        for row in sub_sample:
            row = list(row)
            testingset.append(row)
            actual_label.append(row[-1])
            
        predicted_label = Random_Forest(trainingset, testingset, max_depth, min_size, sample_size, x, no_features)        
        score = checking(predicted_label, actual_label)        
        scores.append(score)
        
    return scores        

In [16]:
# Loading data
import pandas as pd
import csv
df = [] 

with open('pima-indians-diabetes.csv', 'r') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        if not row:
            continue
        
        df.append(row)
len(df)

768

In [17]:
# Hyperparameters
no_data_div = 2    # in how many parts you want to divide your data
max_depth = 30      # maximum depth of tree
min_size = 1       # size of tree
sample_size = 1.0
no_features = int(sqrt(len(df[0])-1))
print(no_features)

x =100 # no_trees, number of trees that you want 


scores = evaluation(df, no_data_div, max_depth, min_size, sample_size, x, no_features)
print('Trees:',x)
print('Scores:',scores)
print('Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

2
Trees: 100
Scores: [63.020833333333336, 67.1875]
Accuracy: 65.104%
